# Dewan Lab Image Analysis

## 0: Run once to create all needed directories at beginning of a project

In [1]:
from Python.DewanLabPython.Helpers import DewanIOhandler
DewanIOhandler.createProjectFramework()

## 1: Always Execute! Load Libraries and User Settings

### 1a: Import Libraries

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
from tqdm import trange


from Python.DewanLabPython.Helpers import DewanDataStore
from Python.DewanLabPython.Helpers import DewanImportDataHelper
from Python.DewanLabPython.Helpers import DewanIOhandler
from Python.DewanLabPython import DewanAUROC
from Python.DewanLabPython import DewanPlotting
from Python.DewanLabPython import DewanStats

### 1b: User Configurables

In [4]:
animal = 'VGAT4'
date = '10-17-22'

# Settings for Inscopix box
LED_power = 1
GAIN = 2.2
FOCUS = 250

PreTrialTime = 3.5  # Imaging time before the final valve opens
PostTrialTime = 3.5  # Imaging time after final valve closes

# Criterion for determining good trials vs bad trials, will be used later
goodCriterion = 0.5
# in the findGoodTrials segment


# Configurables for AUROC
baselineDuration = 2  # number of seconds before the fvOnTime
responseDuration = 2  # number of seconds after the fvOnTime
PlotFigures = True

fileHeader = animal + '-' + date + '-'


## 2: Data Import and Processing

### 2a: Import and pre-process the raw data

In [8]:
# Read in data for processing.  Needs Cell Traces, Odor List, and GPIO file.
folder = './ImagingAnalysis/RawData/'

CellTraceData = pd.read_csv(
    f'{folder}/CellTraces.csv', header=None, skiprows=2, low_memory=False)
OdorData = np.array(pd.read_excel(
    f'{folder}/OdorList.xlsx', usecols=[0], header=None))
GPIOData = pd.read_csv(f'{folder}/GPIO.csv', header=None, low_memory=False)
CellProps = pd.read_csv(f'{folder}/CellTraces-props.csv', header=None)


In [9]:
# Organizes Cell Traces into a numpy array and separates the GPIO information based on indices
# for Sniff, Final Valve etc.
# Converts cell props into a numpy array, ignore the first row of headers
AllCellProps = np.array(CellProps.iloc[1:, :])
# Get indicies where status is not rejected, these are good cells
GoodCells = (AllCellProps[:, 1] != 'rejected')
# Filter the bad cells out of the properites
GoodCellProperties = AllCellProps[GoodCells, :]
# Generate a list of all the possible cell numbers and assign an index
CellList = np.arange(0, len(AllCellProps))
# Filter out all the bad cells; associates cell number with an index
CellList = CellList[GoodCells]
# Insert 1 at the beginning of the truth table, makes filtering bad cells out of trace data easier
GoodCells = np.insert(GoodCells, 0, [1])


CellTraceData = CellTraceData.apply(
    pd.to_numeric, errors='coerce')  # Convert everything to numbers
CellTraceData = np.array(CellTraceData)  # Convert pandas DF to np.array
# Filter out all the bad cell trace data, leaving only the good cells
CellTraceData = CellTraceData[:, GoodCells]
CellTraceData[:, 0] = CellTraceData[:, 0].round(2)  # Round the times to 0.2
GPIOData.iloc[:, 1] = GPIOData.iloc[:, 1].str.replace(' ', '')  # Remove Random Spaces

In [10]:

GPIO1 = np.array(GPIOData.iloc[:, 1] == "GPIO-1")  # Sniff Sensor Data
GPIO2 = np.array(GPIOData.iloc[:, 1] == "GPIO-2")  # FV Actuation Data
# GPIO3 = np.array(GPIOData.iloc[:,1] == "GPIO-3")  # Running Wheel Data
# GPIO4 = np.array(GPIOData.iloc[:,1] == "GPIO-4")  # Lick Data

SniffTable = np.array(GPIOData.iloc[GPIO1,:])
FVData = np.array(GPIOData.iloc[GPIO2,:])

In [11]:
# Pickle the reorganized CellTraceData incase its needed later
# Saves Cell Traces, GPIO, Odor List, Sniff, FV data, Good Cell Properties, and the list of Good Cells

# Once these have been saved, they don't need to be re-run on the same data again unless
# the data itself is changed

folder = './ImagingAnalysis/PreProcessedData'

DewanIOhandler.saveDataToDisk(
    CellTraceData, 'CellTraceData', fileHeader, folder)
DewanIOhandler.saveDataToDisk(GPIOData, 'GPIOData', fileHeader, folder)
DewanIOhandler.saveDataToDisk(OdorData, 'OdorData', fileHeader, folder)
DewanIOhandler.saveDataToDisk(SniffTable, 'SniffTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(FVData, 'FVdata', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    GoodCellProperties, 'GoodCellProperties', fileHeader, folder)
DewanIOhandler.saveDataToDisk(CellList, 'CellList', fileHeader, folder)


VGAT4-10-17-22-CellTraceData has been saved!
VGAT4-10-17-22-GPIOData has been saved!
VGAT4-10-17-22-OdorData has been saved!
VGAT4-10-17-22-SniffTable has been saved!
VGAT4-10-17-22-FVdata has been saved!
VGAT4-10-17-22-GoodCellProperties has been saved!
VGAT4-10-17-22-CellList has been saved!


### 2b: Checkpoint: Load Preprocessed Data

In [12]:
# Opens the saved pickle files.  If the files have already been saved, code can be re-run
# starting from this point
folder = 'ImagingAnalysis\PreProcessedData'

CellTraceData = DewanIOhandler.loadDataFromDisk(
    'CellTraceData', fileHeader, folder)
FVData = DewanIOhandler.loadDataFromDisk('FVData', fileHeader, folder)
GPIOData = DewanIOhandler.loadDataFromDisk('GPIOData', fileHeader, folder)
OdorData = DewanIOhandler.loadDataFromDisk('OdorData', fileHeader, folder)
SniffTable = DewanIOhandler.loadDataFromDisk('SniffTable', fileHeader, folder)
GoodCellProperties = DewanIOhandler.loadDataFromDisk(
    'GoodCellProperties', fileHeader, folder)
CellList = DewanIOhandler.loadDataFromDisk('CellList', fileHeader, folder)


VGAT4-10-17-22-CellTraceData has loaded successfully!
VGAT4-10-17-22-FVData has loaded successfully!
VGAT4-10-17-22-GPIOData has loaded successfully!
VGAT4-10-17-22-OdorData has loaded successfully!
VGAT4-10-17-22-SniffTable has loaded successfully!
VGAT4-10-17-22-GoodCellProperties has loaded successfully!
VGAT4-10-17-22-CellList has loaded successfully!


### 2c: Cell Data Complilation

In [13]:
# Parses the final valve data to identify when the final valve is open vs when it is closed.
FVValue = FVData[:, 2].astype(float)
nValue = len(FVValue)

#nOnOff = -1
valveStatus = 0
FVoffIdx = []
FVonIdx = []
for i in trange((nValue-1), desc="Processing: "):
    vDiff = FVValue[i+1] - FVValue[i]

    if valveStatus == 0:
        if vDiff > 10000:
            #nOnOff += 1
            # print(nOnOff)
            FVonIdx.append(i+1)
            valveStatus = 1
            # print(FVonIdx)
    else:
        if vDiff < -10000:
            FVoffIdx.append(i)
            valveStatus = 0

FVoffIdx = np.array(FVoffIdx)  # verified to match MTLB
FVonIdx = np.array(FVonIdx)  # verified to match MTLB


Processing: 100%|██████████| 7997935/7997935 [00:05<00:00, 1513171.06it/s]


In [15]:
# creates a list of data for the start and end time points of trials
# eliminates bad trials (trials with too short/long of a trial period etc.)
# searches through the start and ends of the good trials for matching timepoints
# and outputs indices for the on/off points of good trials
startTrialTimes = np.array(FVData[FVonIdx, 0].astype(float) - PreTrialTime)  # verified to match MTLB
endTrialTimes = np.array(FVData[FVoffIdx, 0].astype(float) + PostTrialTime)  # verified to match MTLB
CellTraceonIdx, goodTrialsOn = DewanImportDataHelper.findGoodTrials(CellTraceData, startTrialTimes, goodCriterion)  # verified to match MTLB
CellTraceoffIdx, goodTrialsOff = DewanImportDataHelper.findGoodTrials(CellTraceData, endTrialTimes, goodCriterion)  # verified to match MTLB
goodTrials, goodIndOn, goodIndOff = DewanImportDataHelper.intersect_matlab(goodTrialsOn, goodTrialsOff)  # verified to match MTLB


In [16]:
# Index all data by the good trials to exclude data from bad trials

# verified to match MTLB
CellTraceonIdx = CellTraceonIdx[goodIndOn.astype(int)]
# verified to match MTLB
CellTraceoffIdx = CellTraceoffIdx[goodIndOff.astype(int)]

# https://knowyourmeme.com/photos/1402268 ?????
goodIdx = goodTrials.astype(int)-1

FVonIdx = FVonIdx[goodTrials.astype(int)]  # verified to match MTLB
FVoffIdx = FVoffIdx[goodTrials.astype(int)]  # verified to match MTLB
startTrialTimes = startTrialTimes[goodTrials.astype(int)]  # verified to match MTLB
endTrialTimes = endTrialTimes[goodTrials.astype(int)]  # verified to match MTLB
OdorData = OdorData[goodTrials.astype(int)]  # verified to match MTLB


#### Sniffing Data Compilation

In [ ]:
# Compiles data for sniffing from good trialss
sniffStartIdx = np.zeros(FVonIdx.size)
sniffEndIdx = np.zeros(FVonIdx.size)

for i in trange(len(goodTrials), desc="Trial: "):
    startTrial_Time_s_ = float(FVData[:, 0][FVonIdx[i]]) - PreTrialTime
    endTrial_Time_s_ = float(FVData[:, 0][FVoffIdx[i]]) + PostTrialTime
    sniffStartIdx[i] = len(np.nonzero(
        SniffTable[:, 0].astype(float) < startTrial_Time_s_)[0])
    sniffEndIdx[i] = len(np.nonzero(
        SniffTable[:, 0].astype(float) < endTrial_Time_s_)[0]) - 1

nSniffPoints = np.max(sniffEndIdx - sniffStartIdx)

Sniff = np.zeros((nSniffPoints.astype(int), len(sniffStartIdx)))

for numi in trange(len(sniffStartIdx), desc="Trial: "):
    nSniff = sniffEndIdx[numi] - sniffStartIdx[numi]
    Sniff[0:int(nSniff), numi] = SniffTable[sniffStartIdx[numi].astype(
        int):sniffEndIdx[numi].astype(int), 2]

DewanIOhandler.saveDataToDisk(
    Sniff, 'SniffData', fileHeader, 'PreProcessedData')


### 2d: Gather all **cell v. time v. trial** data into single array

In [17]:
nDataPoint = np.max(CellTraceoffIdx - CellTraceonIdx)
Data = np.zeros((len(CellList), len(CellTraceoffIdx), nDataPoint.astype(int)))

for cells in range(len(CellList)):
    for trials in range(len(CellTraceonIdx)):
        nDataPoints = (CellTraceoffIdx[trials] - CellTraceonIdx[trials])
        Data[cells, trials, :nDataPoints[0].astype(int)] = CellTraceData[CellTraceonIdx[trials][0].astype(
            int):CellTraceoffIdx[trials][0].astype(int), cells+1]

Data = np.delete(Data, 180, 2)
Baseline_Shift_Data = Data + abs(np.min(Data))


In [18]:
# Map of trial times relative to startTrial time
FVTimeMap = np.zeros((len(CellTraceoffIdx), nDataPoint.astype(int)))
# Map of times per trial in unix time for AUROC
UnixTimeArray = np.zeros((len(CellTraceoffIdx), max(
    CellTraceoffIdx - CellTraceonIdx).astype(int)[0]))

for trial in range(len(CellTraceoffIdx)):
    nDataPoints = (CellTraceoffIdx[trial] - CellTraceonIdx[trial])
    FVTimeMap[trial, :nDataPoints[0].astype(int)] = CellTraceData[CellTraceonIdx[trial][0].astype(
        int):CellTraceoffIdx[trial][0].astype(int), 0] - float(FVData[FVonIdx[trial], 0])
    UnixTimeArray[trial, :nDataPoints[0].astype(int)] = CellTraceData[CellTraceonIdx[trial][0].astype(
        int):CellTraceoffIdx[trial][0].astype(int), 0]

FVTimeMap = np.delete(FVTimeMap, 180, 1)
UnixTimeArray = np.delete(UnixTimeArray, 180, 1)


### 2e: Combined Data Output

In [22]:
odorIndexes = []

for i in OdorData:
    odorIndexes.append(i[0])

columnNames = ['CentroidX', 'CentroidY', 'NumComponents', 'Size']
TOC = pd.DataFrame(GoodCellProperties[:, 5:9], CellList, columnNames)

path = f'./ImagingAnalysis/CombinedData/{fileHeader}CombinedData.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

TOC.to_excel(writer, sheet_name='TOC')
TimeMapSheet = pd.DataFrame(
    FVTimeMap[:, :180], np.arange(1, len(CellTraceoffIdx)+1))
TimeMapSheet.to_excel(writer, sheet_name='TimeMap')

for i in trange(len(CellList), desc="Writing Cell: "):
    DataDf = pd.DataFrame(Baseline_Shift_Data[i][:][:], odorIndexes)
    #DataDf.index = indexes
    DataDf.to_excel(writer, sheet_name='Cell %d' % CellList[i])

writer.close()

#headers = []

# for i in range(len(Sniff[0,:])):
# headers.append('Trial ' + str(i))

#SniffDF = pd.DataFrame(Sniff)
#SniffDF.to_excel('./ImportOutputs/' + fileHeader + 'SniffData.xlsx', sheet_name = 'Data', header=headers)


Writing Cell: 100%|██████████| 17/17 [00:09<00:00,  1.84it/s]


### 2f: Save information needed for AUROC

In [24]:
DewanIOhandler.saveDataToDisk(Data, 'CombinedData', fileHeader, './ImagingAnalysis/CombinedData')

folder = './ImagingAnalysis/AUROCImports'
DewanIOhandler.saveDataToDisk(FVonIdx, 'FVonIdx', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    UnixTimeArray, 'UnixTimeArray', fileHeader, folder)
DewanIOhandler.saveDataToDisk(FVTimeMap, 'FVTimeMap', fileHeader, folder)
DewanIOhandler.saveDataToDisk(OdorData, 'OdorData', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    Baseline_Shift_Data, 'Baseline_Shift_Data', fileHeader, folder)


VGAT4-10-17-22-CombinedData has been saved!
VGAT4-10-17-22-FVonIdx has been saved!
VGAT4-10-17-22-UnixTimeArray has been saved!
VGAT4-10-17-22-FVTimeMap has been saved!
VGAT4-10-17-22-OdorData has been saved!
VGAT4-10-17-22-Baseline_Shift_Data has been saved!


## 3: AUROC

### 3a: Import Data 

In [26]:
Data = DewanIOhandler.loadDataFromDisk('CombinedData', fileHeader, './ImagingAnalysis/CombinedData')

folder = 'ImagingAnalysis/PreProcessedData'
FVData = DewanIOhandler.loadDataFromDisk('FVData', fileHeader, folder)
CellList = DewanIOhandler.loadDataFromDisk('CellList', fileHeader, folder)

folder = 'ImagingAnalysis/AUROCImports'
FVonIdx = DewanIOhandler.loadDataFromDisk('FVonIdx', fileHeader, folder)
UnixTimeArray = DewanIOhandler.loadDataFromDisk(
    'UnixTimeArray', fileHeader, folder)
OdorData = DewanIOhandler.loadDataFromDisk('OdorData', fileHeader, folder)
FVTimeMap = DewanIOhandler.loadDataFromDisk('FVTimeMap', fileHeader, folder)
Baseline_Shift_Data = DewanIOhandler.loadDataFromDisk(
    'Baseline_Shift_Data', fileHeader, folder)


VGAT4-10-17-22-CombinedData has loaded successfully!
VGAT4-10-17-22-FVData has loaded successfully!
VGAT4-10-17-22-CellList has loaded successfully!
VGAT4-10-17-22-FVonIdx has loaded successfully!
VGAT4-10-17-22-UnixTimeArray has loaded successfully!
VGAT4-10-17-22-OdorData has loaded successfully!
VGAT4-10-17-22-FVTimeMap has loaded successfully!
VGAT4-10-17-22-Baseline_Shift_Data has loaded successfully!


### 3b: Run AUROC

In [27]:
AUROCData = DewanDataStore.AUROCdataStore(
    Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, PlotFigures)
onTimeReturnValues = DewanAUROC.AUROC(AUROCData, False)
latentReturnValues = DewanAUROC.AUROC(AUROCData, True)

In [28]:
onTimeAUROCSignificanceTable = []
onTimeAUROCValueTable = []
onTimeAUROC_Ubounds = []
onTimeAUROC_Lbounds = []
onTimeAUROC_prctile = []


latentAUROCSignificanceTable = []
latentAUROCValueTable = []
latentAUROC_Ubounds = []
latentAUROC_Lbounds = []
latentAUROC_prctile = []

baseline_start_indexes = []
baseline_end_indexes = []
ontime_evoked_start_indexes = []
ontime_evoked_end_indexes = []
latent_evoked_start_indexes = []
latent_evoked_end_indexes = []

for returns in onTimeReturnValues:
    onTimeAUROCSignificanceTable.append(returns.response_chart)
    onTimeAUROCValueTable.append(returns.auroc_values)
    onTimeAUROC_Ubounds.append(returns.all_upper_bounds)
    onTimeAUROC_Lbounds.append(returns.all_lower_bounds)
    onTimeAUROC_prctile.append(returns.percentiles)
    baseline_start_indexes.append(returns.baseline_start_indexes)
    baseline_end_indexes.append(returns.baseline_end_indexes)
    ontime_evoked_start_indexes.append(returns.evoked_start_indexes)
    ontime_evoked_end_indexes.append(returns.evoked_end_indexes)


for returns in latentReturnValues:
    latentAUROCSignificanceTable.append(returns.response_chart)
    latentAUROCValueTable.append(returns.auroc_values)
    latentAUROC_Ubounds.append(returns.all_upper_bounds)
    latentAUROC_Lbounds.append(returns.all_lower_bounds)
    latentAUROC_prctile.append(returns.percentiles)
    latent_evoked_start_indexes.append(returns.evoked_start_indexes)
    latent_evoked_end_indexes.append(returns.evoked_end_indexes)

onTimeAUROCSignificanceTable = np.array(onTimeAUROCSignificanceTable)
latentAUROCSignificanceTable = np.array(latentAUROCSignificanceTable)
onTimeAUROCSignificanceTable = onTimeAUROCSignificanceTable.astype(float)
latentAUROCSignificanceTable = latentAUROCSignificanceTable.astype(float)

#baseline_start_indexes = np.array(baseline_start_indexes)
# baseline_end_indexes = np.array(baseline_end_indexes)
# ontime_evoked_start_indexes = np.array(ontime_evoked_start_indexes)
# ontime_evoked_end_indexes = np.array(ontime_evoked_end_indexes)
# latent_evoked_start_indexes = np.array(latent_evoked_start_indexes)
# latent_evoked_end_indexes = np.array(latent_evoked_end_indexes)

for rows in range(len(latentAUROCSignificanceTable)):
    evokedCells = np.nonzero(onTimeAUROCSignificanceTable[rows] != 0)
    latentAUROCSignificanceTable[rows][evokedCells] = 0

### 3c: Save AROC

In [29]:
folder = 'ImagingAnalysis/AUROCData'

DewanIOhandler.saveDataToDisk(
    onTimeAUROCSignificanceTable, 'onTimeAUROCSignificanceTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROCValueTable, 'onTimeAUROCValueTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROC_Lbounds, 'onTimeAUROC_Lbounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROC_Ubounds, 'onTimeAUROC_Ubounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROC_prctile, 'onTimeAUROC_prctile', fileHeader, folder)

DewanIOhandler.saveDataToDisk(
    baseline_start_indexes, 'baseline_start_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    baseline_end_indexes, 'baseline_end_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    ontime_evoked_start_indexes, 'ontime_evoked_start_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    ontime_evoked_end_indexes, 'ontime_evoked_end_indexes', fileHeader, folder)

DewanIOhandler.saveDataToDisk(
    latentAUROCSignificanceTable, 'latentAUROCSignificanceTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROCValueTable, 'latentAUROCValueTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROC_Lbounds, 'latentAUROC_Lbounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROC_Ubounds, 'latentAUROC_Ubounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROC_prctile, 'latentAUROC_prctile', fileHeader, folder)

DewanIOhandler.saveDataToDisk(
    latent_evoked_start_indexes, 'latent_evoked_start_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latent_evoked_end_indexes, 'latent_evoked_end_indexes', fileHeader, folder)

VGAT4-10-17-22-onTimeAUROCSignificanceTable has been saved!
VGAT4-10-17-22-onTimeAUROCValueTable has been saved!
VGAT4-10-17-22-onTimeAUROC_Lbounds has been saved!
VGAT4-10-17-22-onTimeAUROC_Ubounds has been saved!
VGAT4-10-17-22-onTimeAUROC_prctile has been saved!
VGAT4-10-17-22-baseline_start_indexes has been saved!
VGAT4-10-17-22-baseline_end_indexes has been saved!
VGAT4-10-17-22-ontime_evoked_start_indexes has been saved!
VGAT4-10-17-22-ontime_evoked_end_indexes has been saved!
VGAT4-10-17-22-latentAUROCSignificanceTable has been saved!
VGAT4-10-17-22-latentAUROCValueTable has been saved!
VGAT4-10-17-22-latentAUROC_Lbounds has been saved!
VGAT4-10-17-22-latentAUROC_Ubounds has been saved!
VGAT4-10-17-22-latentAUROC_prctile has been saved!
VGAT4-10-17-22-latent_evoked_start_indexes has been saved!
VGAT4-10-17-22-latent_evoked_end_indexes has been saved!


## 4: Plotting

### 4a: Import Data

In [5]:
folder = 'ImagingAnalysis/AUROCData'

onTimeAUROCSignificanceTable = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROCSignificanceTable', fileHeader, folder)
onTimeAUROCValueTable = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROCValueTable', fileHeader, folder)
onTimeAUROC_Lbounds = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROC_Lbounds', fileHeader, folder)
onTimeAUROC_Ubounds = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROC_Ubounds', fileHeader, folder)
onTimeAUROC_prctile = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROC_prctile', fileHeader, folder)

latentAUROCSignificanceTable = DewanIOhandler.loadDataFromDisk(
    'latentAUROCSignificanceTable', fileHeader, folder)
latentAUROCValueTable = DewanIOhandler.loadDataFromDisk(
    'latentAUROCValueTable', fileHeader, folder)
latentAUROC_Lbounds = DewanIOhandler.loadDataFromDisk(
    'latentAUROC_Lbounds', fileHeader, folder)
latentAUROC_Ubounds = DewanIOhandler.loadDataFromDisk(
    'latentAUROC_Ubounds', fileHeader, folder)
latentAUROC_prctile = DewanIOhandler.loadDataFromDisk(
    'latentAUROC_prctile', fileHeader, folder)


baseline_start_indexes = DewanIOhandler.loadDataFromDisk(
    'baseline_start_indexes', fileHeader, folder)
baseline_end_indexes = DewanIOhandler.loadDataFromDisk(
    'baseline_end_indexes', fileHeader, folder)
ontime_evoked_start_indexes = DewanIOhandler.loadDataFromDisk(
    'ontime_evoked_start_indexes', fileHeader, folder)
ontime_evoked_end_indexes = DewanIOhandler.loadDataFromDisk(
    'ontime_evoked_end_indexes', fileHeader, folder)
latent_evoked_start_indexes = DewanIOhandler.loadDataFromDisk(
    'latent_evoked_start_indexes', fileHeader, folder)
latent_evoked_end_indexes = DewanIOhandler.loadDataFromDisk(
    'latent_evoked_end_indexes', fileHeader, folder)

Data = DewanIOhandler.loadDataFromDisk(
    'CombinedData', fileHeader, 'ImagingAnalysis/CombinedData')

folder = 'ImagingAnalysis/PreProcessedData'
FVData = DewanIOhandler.loadDataFromDisk('FVData', fileHeader, folder)
CellList = DewanIOhandler.loadDataFromDisk('CellList', fileHeader, folder)

folder = 'ImagingAnalysis/AUROCImports'
FVonIdx = DewanIOhandler.loadDataFromDisk('FVonIdx', fileHeader, folder)
UnixTimeArray = DewanIOhandler.loadDataFromDisk(
    'UnixTimeArray', fileHeader, folder)
OdorData = DewanIOhandler.loadDataFromDisk('OdorData', fileHeader, folder)
FVTimeMap = DewanIOhandler.loadDataFromDisk('FVTimeMap', fileHeader, folder)
Baseline_Shift_Data = DewanIOhandler.loadDataFromDisk(
    'Baseline_Shift_Data', fileHeader, folder)


VGAT4-10-17-22-onTimeAUROCSignificanceTable has loaded successfully!
VGAT4-10-17-22-onTimeAUROCValueTable has loaded successfully!
VGAT4-10-17-22-onTimeAUROC_Lbounds has loaded successfully!
VGAT4-10-17-22-onTimeAUROC_Ubounds has loaded successfully!
VGAT4-10-17-22-onTimeAUROC_prctile has loaded successfully!
VGAT4-10-17-22-latentAUROCSignificanceTable has loaded successfully!
VGAT4-10-17-22-latentAUROCValueTable has loaded successfully!
VGAT4-10-17-22-latentAUROC_Lbounds has loaded successfully!
VGAT4-10-17-22-latentAUROC_Ubounds has loaded successfully!
VGAT4-10-17-22-latentAUROC_prctile has loaded successfully!
VGAT4-10-17-22-baseline_start_indexes has loaded successfully!
VGAT4-10-17-22-baseline_end_indexes has loaded successfully!
VGAT4-10-17-22-ontime_evoked_start_indexes has loaded successfully!
VGAT4-10-17-22-ontime_evoked_end_indexes has loaded successfully!
VGAT4-10-17-22-latent_evoked_start_indexes has loaded successfully!
VGAT4-10-17-22-latent_evoked_end_indexes has loaded 

### 4b: Cells v. Odors AUROC Significance

In [6]:
onTimePlottingData = DewanDataStore.PlottingDataStore(Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVTimeMap, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, onTimeAUROCSignificanceTable, onTimeAUROC_Lbounds, onTimeAUROC_Ubounds, onTimeAUROC_prctile, baseline_start_indexes, baseline_end_indexes, ontime_evoked_start_indexes, ontime_evoked_end_indexes)

latentPlottingData = DewanDataStore.PlottingDataStore(Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVTimeMap, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, latentAUROCSignificanceTable, latentAUROC_Lbounds, latentAUROC_Ubounds, latentAUROC_prctile, baseline_start_indexes, baseline_end_indexes, latent_evoked_start_indexes, latent_evoked_end_indexes)

#### 4b-1: Plot Cell v Odor Significance Matricies

In [32]:
DewanPlotting.plotCellvOdorMatricies(onTimePlottingData, False)
DewanPlotting.plotCellvOdorMatricies(latentPlottingData, True)

#### 4b-2: Plot All Cell Traces

In [8]:
# Plot all cells v odors; only needed for AUROC verification
DewanPlotting.plotAllCells(onTimePlottingData, False)
DewanPlotting.plotAllCells(latentPlottingData, True)

#### 4b-3: Plot Significant Cell Traces

In [ ]:
# Oh no! Nothing here yet :(
# https://media.tenor.com/vMvR0gEolCEAAAAC/nothing-to-see-here-explosion.gif

### 4c: Plot Trace Variance

In [32]:
trialScatterData = DewanDataStore.AUROCdataStore(
    Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, False)


DewanPlotting.plotTrialsPerPairing(
    trialScatterData, onTimeAUROCSignificanceTable, False)
DewanPlotting.plotTrialsPerPairing(
    trialScatterData, onTimeAUROCSignificanceTable, True)


## 5. Statistics

### 5a. Load Data

In [ ]:
#Oh no, nothing here yet :(

### 5b. Correlation Coefficients

In [37]:
similarityData = DewanDataStore.AUROCdataStore(Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, False)
CCs = DewanStats.crossTrialConsistency(similarityData, onTimeAUROCSignificanceTable, False)

In [38]:
mineralOilIndex = np.nonzero(similarityData.unique_odors == 'MO')[0]
responsiveCellListIndex = np.nonzero(onTimeAUROCSignificanceTable[:, mineralOilIndex] == 0)[0]
odorIndexes = np.nonzero(similarityData.unique_odors != 'MO')[0]
columns = [f'Cell: {similarityData.Cell_List[cell]}' for cell in responsiveCellListIndex]
rows = [f'{similarityData.unique_odors[odor]}' for odor in odorIndexes]
CCs = np.array(CCs)
CCs = pd.DataFrame(CCs.transpose(), columns=columns, index=rows)

In [39]:
CCs

,Cell: 0,Cell: 1,Cell: 2,Cell: 3,Cell: 9,Cell: 10,Cell: 11,Cell: 14,Cell: 15,Cell: 16,Cell: 17,Cell: 19,Cell: 22,Cell: 24,Cell: 26,Cell: 43
25ONE,0.198985,0.085805,0.134829,0.052483,0.121492,0.054131,0.195258,0.203805,0.177296,0.072994,0.240276,0.020716,0.123642,0.205818,0.201067,0.199051
4AL,0.194291,0.135975,0.173829,0.186535,0.151909,0.212283,0.131235,0.171773,0.142373,0.143611,0.344949,0.218073,0.118897,0.214641,0.256203,0.200011
4AMINE,0.210934,0.163678,0.154829,0.184841,0.106230,0.081718,0.193497,0.283039,0.223111,0.184213,0.441769,0.291145,0.265386,0.179967,0.193423,0.182966
4ATE,0.243845,0.221955,0.200309,0.165513,0.216327,0.231058,0.219674,0.169526,0.165077,0.223559,0.164941,0.138637,0.219203,0.245240,0.213648,0.227447
4OL,0.266180,0.188068,0.301149,0.212138,0.218819,0.228408,0.267794,0.256187,0.276194,0.160607,0.231869,0.165822,0.193592,0.272203,0.249222,0.221420
4ONE,0.229144,0.206167,0.274633,0.238270,0.294731,0.257804,0.247274,0.162846,0.271217,0.286897,0.237308,0.196246,0.162536,0.227232,0.263294,0.227026
5AL,0.209614,0.265450,0.204649,0.200445,0.150014,0.241715,0.234240,0.247971,0.185293,0.232157,0.259041,0.141620,0.228766,0.264915,0.247534,0.185829
5AMINE,0.226623,0.189088,0.206849,0.191508,0.107316,0.038510,0.235872,0.274041,0.202881,0.145923,0.395755,0.209216,0.298810,0.190645,0.248349,0.236197
5ATE,0.192513,0.228214,0.223776,0.236023,0.153958,0.178395,0.245132,0.159229,0.170200,0.178313,0.222572,0.053045,0.106605,0.246769,0.257851,0.250190
5OL,0.243577,0.150894,0.199629,0.256055,0.160150,0.173721,0.208160,0.267014,0.236400,0.183825,0.218780,0.073640,0.100864,0.247138,0.267280,0.244423


### 5c. Lifetime and Population Sparseness

In [40]:
sparsenessData = DewanDataStore.AUROCdataStore(
    Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FVonIdx, UnixTimeArray, baselineDuration, responseDuration, False)

lifetimeSparseness, cells = DewanStats.lifetimeSparseness(sparsenessData, onTimeAUROCSignificanceTable)
populationSparseness, odors = DewanStats.popSparseness(sparsenessData, onTimeAUROCSignificanceTable)
lsPairs = [f'Cell {cells[i]}: {round(lifetimeSparseness[i],5)}' for i in range(len(cells))]
psPairs = [f'Odor: {sparsenessData.unique_odors[odors[i]]}: {round(populationSparseness[i],5)}' for i in range(len(odors))]

AttributeError: 'bool' object has no attribute 'baseline_start_indexes'

In [49]:
print(psPairs)

['Odor: 25ONE: 1.03987', 'Odor: 4AL: 0.98182', 'Odor: 4AMINE: 0.86699', 'Odor: 4ATE: 1.04437', 'Odor: 4OL: 0.99882', 'Odor: 4ONE: 0.96804', 'Odor: 5AL: 1.04548', 'Odor: 5AMINE: 0.88918', 'Odor: 5ATE: 1.04753', 'Odor: 5OL: 1.00393', 'Odor: 6AL: 1.02612', 'Odor: 6AMINE: 0.96569', 'Odor: 6ATE: 0.85856', 'Odor: 6OL: 0.96257', 'Odor: 6ONE: 0.98143', 'Odor: 7AL: 1.01253', 'Odor: 7AMINE: 0.93744', 'Odor: 7ATE: 0.94598', 'Odor: 7OL: 0.98057', 'Odor: 7ONE: 1.03446', 'Odor: Buzzer: 1.03013']


In [41]:
mineralOilIndex = np.nonzero(sparsenessData.unique_odors == 'MO')[0]
odorIndexes = np.nonzero(sparsenessData.unique_odors != 'MO')[0]

In [ ]:
print(odorIndexes)
